In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os
from itertools import product
from functions import (load_parameters, load_generation_data, load_price_data, generate_randomized_generation,
generate_rt_scenarios, plot_generation_data, plot_randomized_generation, plot_scenarios_for_generator, plot_rt_scenarios)

generation_data, I, T = load_generation_data(date_filter="2022-07-18")
S, R, P_RT_v, K, K0, M1, M2 = load_parameters(I, T, generation_data)
P_DA, P_PN = load_price_data()

✅ 총 3개 파일을 불러왔습니다: 1201.csv, 524.csv, 89.csv
📊 데이터 Shape: I=3, T=24, S=20
✅ 시뮬레이션 초기화 완료: S=20, Randomness='high', M1=607.34, M2=1066.74


In [28]:
# models
only = gp.Model("only")
only.Params.OutputFlag = 0
only.setParam("MIPGap", 1e-7)

x = only.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
yp = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
ym = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
zeta = only.addVars(I, T, S, vtype=GRB.BINARY, name="z")
only.update()

obj = gp.quicksum(P_DA[t] * x[i, t] for i in range(I) for t in range(T)) + gp.quicksum(
    1/S * (P_RT_v[t, s] * yp[i, t, s] - P_PN[t] * ym[i, t, s]) for i in range(I) for t in range(T) for s in range(S))
only.setObjective(obj, GRB.MAXIMIZE)

for i in range(I):
    for t in range(T):
        for s in range(S):
            only.addConstr(R[i, t, s] - x[i, t] == yp[i, t, s] - ym[i, t, s])
            only.addConstr(yp[i, t, s] <= R[i, t, s])
            only.addConstr(yp[i, t, s] <= M1 * zeta[i, t, s])
            only.addConstr(ym[i, t, s] <= M1 * (1 - zeta[i, t, s]))

only.optimize()

if only.status == GRB.OPTIMAL:
    print(f"[ONLY] {only.objVal}")
else:
    print("No optimal solution found.")
    
# ----------------------------------

agg = gp.Model("agg")
agg.Params.OutputFlag = 0
agg.setParam("MIPGap", 1e-7)

a = agg.addVars(T, vtype=GRB.CONTINUOUS, lb=0, name="alpha")
bp = agg.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_plus")
bm = agg.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_minus")
mu = agg.addVars(T, S, vtype=GRB.BINARY, name="z")
agg.update()

obj = gp.quicksum(P_DA[t] * a[t] for t in range(T)) + gp.quicksum(
    1/S * (P_RT_v[t, s] * bp[t, s] - P_PN[t] * bm[t, s]) for t in range(T) for s in range(S))
agg.setObjective(obj, GRB.MAXIMIZE)

for t in range(T):
    for s in range(S):
        agg.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) - a[t] == bp[t, s] - bm[t, s])
        agg.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) >= bp[t, s])
        agg.addConstr(bp[t, s] <= M2 * mu[t, s])
        agg.addConstr(bm[t, s] <= M2 * (1 - mu[t, s]))
agg.optimize()

if agg.status == GRB.OPTIMAL:
    print(f"[AGG] {agg.objVal}")
else:
    print("No optimal solution found.")

[ONLY] 832414.7880581587
[AGG] 841268.0150591594


In [29]:
x_v   = np.array([[only.getVarByName(f"x[{i},{t}]").X for t in range(T)] for i in range(I)])
yp_v = np.array([[[only.getVarByName(f"y_plus[{i},{t},{s}]").X for s in range(S)] for t in range(T)] for i in range(I)])
ym_v = np.array([[[only.getVarByName(f"y_minus[{i},{t},{s}]").X for s in range(S)] for t in range(T)] for i in range(I)])
a_v  = np.array([agg.getVarByName(f"alpha[{t}]").X for t in range(T)])
bp_v = np.array([[agg.getVarByName(f"beta_plus[{t},{s}]").X for s in range(S)] for t in range(T)])
bm_v = np.array([[agg.getVarByName(f"beta_minus[{t},{s}]").X for s in range(S)] for t in range(T)])

idx = 15

xv   = x_v
ypv  = yp_v[:, :, idx]
ymv  = ym_v[:, :, idx]
av   = a_v
bpv  = bp_v[:, idx]
bmv  = bm_v[:, idx]
P_RT = P_RT_v[:, idx]

In [30]:
delta_DA = np.zeros(T)
delta_BMp = np.zeros(T)
delta_BMm = np.zeros(T)
lambd = np.zeros((I, T))
total_surplus = np.zeros(T)

for t in range(T):
    delta_DA[t]  = av[t]  - np.sum(xv[:, t])
    delta_BMp[t] = bpv[t] - np.sum(ypv[:, t])
    delta_BMm[t] = bmv[t] - np.sum(ymv[:, t])
    total_surplus[t]   = P_DA[t] * delta_DA[t] + P_RT[t] * delta_BMp[t] - P_PN[t] * delta_BMm[t]

print(f"{'t':>2} | {'ΔDA':>10} | {'ΔBM+':>12} | {'ΔBM-':>12} || {'Surplus':>10}")
print("-" * 60)
for t in range(T):
    print(f"{t:2} | {delta_DA[t]:10.4f} | {delta_BMp[t]:12.4f} | {delta_BMm[t]:12.4f} || {total_surplus[t]:10.4f}")
print("-" * 60)
print(f"{'Σ':>2} | {'':>10} | {'':>12} | {'':>12} || {total_surplus.sum():10.4f}")

 t |        ΔDA |         ΔBM+ |         ΔBM- ||    Surplus
------------------------------------------------------------
 0 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 1 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 2 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 3 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 4 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 5 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 6 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 7 |     0.0613 |      -0.0976 |      -0.0362 ||     1.0391
 8 |     4.7020 |      -5.2401 |      -0.5381 ||   -75.4623
 9 |     0.4090 |      -0.4090 |       0.0000 ||     0.1179
10 |    23.4797 |     -23.4797 |       0.0000 ||  -379.8504
11 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
12 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
13 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
14 |    70.6724 |    -161.0324 |     -9

In [31]:
Po        = xv + ypv - ymv                         
Po_prime  = xv + ypv - ymv
delta_Po  = Po - Po_prime
delta_Pop = np.where(delta_Po > 0, delta_Po, 0)
delta_Pom = np.where(delta_Po < 0, -delta_Po, 0)

varphi_p = ypv + delta_Pop
varphi_m = ymv + delta_Pom

print(f"{'i':>2} {'t':>2} | {'y⁺':>8} | {'ΔP⁺':>8} | {'φ⁺':>8} || {'y⁻':>8} | {'ΔP⁻':>8} | {'φ⁻':>8}")
print("-" * 75)
for i in range(I):
    for t in range(T):
        print(f"{i:2} {t:2} | {ypv[i,t]:8.2f} | {delta_Pop[i,t]:8.2f} | {varphi_p[i,t]:8.2f} || {ymv[i,t]:8.2f} | {delta_Pom[i,t]:8.2f} | {varphi_m[i,t]:8.2f}")

 i  t |       y⁺ |      ΔP⁺ |       φ⁺ ||       y⁻ |      ΔP⁻ |       φ⁻
---------------------------------------------------------------------------
 0  0 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  1 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  2 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  3 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  4 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  5 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  6 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  7 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  8 |    19.04 |     0.00 |    19.04 ||     0.00 |     0.00 |     0.00
 0  9 |     3.77 |     0.00 |     3.77 ||     0.00 |     0.00 |     0.00
 0 10 |    22.15 |     0.00 |    22.15 ||     0.00 |     0.00 |     0.00
 0 11 |    73.28 |     0.00 |    73.28 ||     0.

In [32]:
delta_DAp = np.where(delta_DA > 0,  delta_DA, 0)
delta_DAm = np.where(delta_DA < 0, -delta_DA, 0)
print(f"{'t':>2} | {'α':>6} | {'∑x':>6} | {'ΔDA':>8} | {'ΔDA+':>8} | {'ΔDA-':>8}")
print("-" * 55)
for t in range(T):
    print(f"{t:2} | {av[t]:6.2f} | {np.sum(xv[:,t]):6.2f} | {delta_DA[t]:8.2f} | {delta_DAp[t]:8.2f} | {delta_DAm[t]:8.2f}")

 t |      α |     ∑x |      ΔDA |     ΔDA+ |     ΔDA-
-------------------------------------------------------
 0 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 1 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 2 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 3 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 4 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 5 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 6 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 7 |   0.88 |   0.82 |     0.06 |     0.06 |     0.00
 8 |  12.34 |   7.64 |     4.70 |     4.70 |     0.00
 9 |  33.27 |  32.86 |     0.41 |     0.41 |     0.00
10 |  96.16 |  72.68 |    23.48 |    23.48 |     0.00
11 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
12 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
13 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
14 | 585.82 | 515.14 |    70.67 |    70.67 |     0.00
15 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
16 | 248.28 | 232.45 |    

In [33]:
E_INp = np.zeros(T)
E_INm = np.zeros(T)

for t in range(T):
    E_INp[t] = np.sum(varphi_p[:, t]) - (bpv[t] + delta_DAp[t])
    E_INm[t] = np.sum(varphi_m[:, t]) - (bmv[t] + delta_DAm[t])
    
print(f"{'t':>2} | {'∑φ⁺':>9} | {'β⁺':>9} | {'ΔDA+':>7} | {'E_IN⁺':>8} || {'∑φ⁻':>7} | {'β⁻':>6} | {'ΔDA-':>6} | {'E_IN⁻':>7}")
print("-" * 90)
for t in range(T):
    sum_phi_p = np.sum(varphi_p[:, t])
    sum_phi_m = np.sum(varphi_m[:, t])
    print(f"{t:2} | {sum_phi_p:9.2f} | {bpv[t]:9.2f} | {delta_DAp[t]:7.2f} | {E_INp[t]:8.2f} ||"
          f" {sum_phi_m:7.2f} | {bmv[t]:6.2f} | {delta_DAm[t]:6.2f} | {E_INm[t]:7.2f}")

 t |       ∑φ⁺ |        β⁺ |    ΔDA+ |    E_IN⁺ ||     ∑φ⁻ |     β⁻ |   ΔDA- |   E_IN⁻
------------------------------------------------------------------------------------------
 0 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 1 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 2 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 3 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 4 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 5 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 6 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 7 |      3.00 |      2.91 |    0.06 |     0.04 ||    0.04 |   0.00 |   0.00 |    0.04
 8 |     19.57 |     14.33 |    4.70 |     0.54 ||    0.54 |   0.00 |   0.00 |    0.54
 9 |     21.29 |     20.88 |    0.41 | 

In [34]:
m = 1e-16
e_INp = np.zeros((I, T))
e_INm = np.zeros((I, T))

for t in range(T):
    for i in range(I):
        e_INp[i, t] = (varphi_p[i, t] / (np.sum(varphi_p[:, t]) + m)) * E_INp[t]
        e_INm[i, t] = (varphi_m[i, t] / (np.sum(varphi_m[:, t]) + m)) * E_INm[t]

print(f"{'i':>2} {'t':>2} | {'φ⁺':>8} | {'∑φ⁺':>8} | {'e_IN⁺':>10} || {'φ⁻':>8} | {'∑φ⁻':>8} | {'e_IN⁻':>10}")
print("-" * 80)
for t in range(20, 23):
    for i in range(I):
        sum_phi_p = np.sum(varphi_p[:, t])
        sum_phi_m = np.sum(varphi_m[:, t])
        print(f"{i:2} {t:2} | {varphi_p[i, t]:8.2f} | {sum_phi_p:8.2f} | {e_INp[i, t]:10.2f} ||"
              f" {varphi_m[i, t]:8.2f} | {sum_phi_m:8.2f} | {e_INm[i, t]:10.2f}")

 i  t |       φ⁺ |      ∑φ⁺ |      e_IN⁺ ||       φ⁻ |      ∑φ⁻ |      e_IN⁻
--------------------------------------------------------------------------------
 0 20 |     0.00 |     7.78 |       0.00 ||     5.41 |     5.41 |       0.87
 1 20 |     0.00 |     7.78 |       0.00 ||     0.00 |     5.41 |       0.00
 2 20 |     7.78 |     7.78 |       0.87 ||     0.00 |     5.41 |       0.00
 0 21 |     3.38 |     3.38 |       0.88 ||     0.00 |     0.88 |       0.00
 1 21 |     0.00 |     3.38 |       0.00 ||     0.88 |     0.88 |       0.88
 2 21 |     0.00 |     3.38 |       0.00 ||     0.00 |     0.88 |       0.00
 0 22 |     0.00 |     0.00 |       0.00 ||     0.00 |     0.00 |       0.00
 1 22 |     0.00 |     0.00 |       0.00 ||     0.00 |     0.00 |       0.00
 2 22 |     0.00 |     0.00 |       0.00 ||     0.00 |     0.00 |       0.00


In [35]:
delta_INp = np.zeros(T)
delta_INm = np.zeros(T)
Lambda_IN = np.zeros(T)
rho_IN = np.zeros(T)

for t in range(T):
    
    if delta_BMp[t] < 0 and E_INp[t] > 1e-6:
        delta_INp[t] = min(E_INp[t], -delta_BMp[t])
    else:
        delta_INp[t] = 0
    
    if delta_BMm[t] < 0 and E_INm[t] > 1e-6:
        delta_INm[t] = min(E_INm[t], -delta_BMm[t])
    else:
        delta_INm[t] = 0
        
    Lambda_IN[t] = P_RT[t] * delta_INp[t] - P_PN[t] * delta_INm[t]
    rho_IN[t] = Lambda_IN[t] / (E_INp[t] + E_INm[t] + m)
    
print(f"{'t':>2} | {'ΔBM+':>7} | {'E_IN+':>7} | {'ΔIN+':>7} || "
      f"{'ΔBM-':>7} | {'E_IN-':>7} | {'ΔIN-':>7} || "
      f"{'Λ_IN':>8} | {'ρ_IN':>8}")
print("-" * 90)

for t in range(T):
    print(f"{t:2} | {delta_BMp[t]:7.2f} | {E_INp[t]:7.2f} | {delta_INp[t]:7.2f} || "
          f"{delta_BMm[t]:7.2f} | {E_INm[t]:7.2f} | {delta_INm[t]:7.2f} || "
          f"{Lambda_IN[t]:8.2f} | {rho_IN[t]:8.2f}")

 t |    ΔBM+ |   E_IN+ |    ΔIN+ ||    ΔBM- |   E_IN- |    ΔIN- ||     Λ_IN |     ρ_IN
------------------------------------------------------------------------------------------
 0 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 1 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 2 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 3 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 4 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 5 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 6 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 7 |   -0.10 |    0.04 |    0.04 ||   -0.04 |    0.04 |    0.04 ||    -1.47 |   -20.26
 8 |   -5.24 |    0.54 |    0.54 ||   -0.54 |    0.54 |    0.54 ||   -19.63 |   -18.24
 9 |   -0.41 |   -0.00 |    0.00 ||    

In [36]:
Lambda_b = np.zeros(T)
rho_b = np.zeros(T)
delta_bp = np.zeros(T)
delta_bm = np.zeros(T)
e_bp = np.zeros((I, T))
e_bm = np.zeros((I, T))

for t in range(T):
    delta_bp[t] = delta_BMp[t] - delta_INp[t]
    delta_bm[t] = delta_BMm[t] - delta_INm[t]
    Lambda_b[t] = (P_RT[t] - P_DA[t]) * delta_bp[t] - (P_PN[t] - P_DA[t]) * delta_bm[t]
    
    e_bp[:, t] = varphi_p[:, t] - e_INp[:, t]
    e_bm[:, t] = varphi_m[:, t] - e_INm[:, t]
    
    total_b = np.sum(e_bp[:, t] + e_bm[:, t]+m)

    if total_b < 1e-10:
        rho_b[t] = 0
    # Lambda_b[t]는 유지 (반영되지 않음)
    else:
        rho_b[t] = Lambda_b[t] / (total_b + m)
    
print(f"{'i':>2} {'t':>2} | {'φ⁺':>7} | {'e_IN⁺':>7} | {'e_b+':>9} || "
      f"{'φ⁻':>7} | {'e_IN⁻':>7} | {'e_b-':>9} || {'Λ_b':>10} | {'ρ_b':>8}")
print("-" * 120)

for i in range(I):
    for t in range(20, 22):
        print(f"{i:2} {t:2} | "
              f"{varphi_p[i, t]:7.4f} | {e_INp[i, t]:7.4f} | {e_bp[i, t]:9.4f} || "
              f"{varphi_m[i, t]:7.4f} | {e_INm[i, t]:7.4f} | {e_bm[i, t]:9.4f} || "
              f"{Lambda_b[t]:10.2f} | {rho_b[t]:8.2f}")

 i  t |      φ⁺ |   e_IN⁺ |      e_b+ ||      φ⁻ |   e_IN⁻ |      e_b- ||        Λ_b |      ρ_b
------------------------------------------------------------------------------------------------------------------------
 0 20 |  0.0000 |  0.0000 |    0.0000 ||  5.4111 |  0.8736 |    4.5376 ||     397.35 |    34.71
 0 21 |  3.3764 |  0.8773 |    2.4990 ||  0.0000 |  0.0000 |    0.0000 ||     350.05 |   140.07
 1 20 |  0.0000 |  0.0000 |    0.0000 ||  0.0000 |  0.0000 |    0.0000 ||     397.35 |    34.71
 1 21 |  0.0000 |  0.0000 |    0.0000 ||  0.8773 |  0.8773 |    0.0000 ||     350.05 |   140.07
 2 20 |  7.7842 |  0.8736 |    6.9106 ||  0.0000 |  0.0000 |    0.0000 ||     397.35 |    34.71
 2 21 |  0.0000 |  0.0000 |    0.0000 ||  0.0000 |  0.0000 |    0.0000 ||     350.05 |   140.07


In [37]:
lambda_it = np.zeros((I, T))
lambda_sum = np.zeros(T)

for t in range(T):
    for i in range(I):
        lambda_it[i, t] += rho_IN[t] * (e_INp[i, t] + e_INm[i, t])
        lambda_it[i, t] += rho_b[t] * (e_bp[i, t] + e_bm[i, t])
    lambda_sum[t] = np.sum(lambda_it[:, t])     

print(f"{'i':>2} {'t':>2} | {'λ_it':>10}")
print("-" * 20)
for t, i in product(range(20, 23), range(I)):
    print(f"{i:2} {t:2} | {lambda_it[i, t]:10.4f}")

 i  t |       λ_it
--------------------
 0 20 |   115.4242
 1 20 |     0.0000
 2 20 |   197.7890
 0 21 |   298.0358
 1 21 |   -52.0133
 2 21 |     0.0000
 0 22 |     0.0000
 1 22 |     0.0000
 2 22 |     0.0000


In [38]:
print(f"{'t':>2} | {'Λ_IN':>8} | {'Λ_b':>8} || {'∑λ_it':>10} | {'Difference':>10}")
print("-" * 65)

total_IN = 0
total_b  = 0
total_lambda = 0

for t in range(T):
    total_lhs = Lambda_IN[t] + Lambda_b[t]
    diff = total_lhs - lambda_sum[t]

    # 누적합 계산
    total_IN     += Lambda_IN[t]
    total_b      += Lambda_b[t]
    total_lambda += lambda_sum[t]

    print(f"{t:2} | {Lambda_IN[t]:8.2f} | {Lambda_b[t]:8.2f} || "
          f"{lambda_sum[t]:10.2f} | {diff:10.2f}")

# 마지막 줄: 총합 정리
print("-" * 65)
print(f"{'Σ':>2} | {total_IN:8.2f} | {total_b:8.2f}|| "
      f"{total_lambda:10.2f} | {total_IN + total_b - total_lambda:10.2f}")

 t |     Λ_IN |      Λ_b ||      ∑λ_it | Difference
-----------------------------------------------------------------
 0 |     0.00 |    -0.00 ||       0.00 |       0.00
 1 |     0.00 |     0.00 ||       0.00 |       0.00
 2 |     0.00 |    -0.00 ||       0.00 |       0.00
 3 |     0.00 |    -0.00 ||       0.00 |       0.00
 4 |     0.00 |    -0.00 ||       0.00 |       0.00
 5 |     0.00 |     0.00 ||       0.00 |       0.00
 6 |     0.00 |    -0.00 ||       0.00 |       0.00
 7 |    -1.47 |     2.51 ||       1.04 |      -0.00
 8 |   -19.63 |   -55.83 ||     -75.46 |       0.00
 9 |     0.00 |     0.12 ||       0.12 |       0.00
10 |     0.00 |  -379.85 ||    -379.85 |       0.00
11 |     0.00 |     0.00 ||       0.00 |       0.00
12 |     0.00 |     0.00 ||       0.00 |       0.00
13 |     0.00 |     0.00 ||       0.00 |       0.00
14 | -9194.10 | 19996.37 ||   10802.27 |      -0.00
15 |     0.00 |     0.00 ||       0.00 |       0.00
16 | -2236.30 |  5016.39 ||    2780.09 |      -0.0

In [39]:
total_surplus.sum()

13670.668944536807